<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

## Data Load & Prep
In this notebook we show how to download and preprocess the [STS Benchmark](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) dataset for sentence similarity.

For this task, the only preprocessing we need to do is

* Make all text lowercase
* Tokenize (segment the text into words, punctuation marks, etc). Here we show an example using [spaCy](https://spacy.io/)

### 00 Global Settings

In [1]:
import sys

sys.path.append("../../")  ## set the environment path

import os
import azureml.dataprep as dp
import pandas as pd

from utils_nlp.dataset import stsbenchmark
from utils_nlp.dataset.preprocess import to_lowercase, to_spacy_tokens
from utils_nlp.dataset.url_utils import maybe_download
from utils_nlp.dataset.preprocess import rm_spacy_stopwords

print("System version: {}".format(sys.version))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
STS_URL = "http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz"
BASE_DATA_PATH = "../../data"
RAW_DATA_PATH = os.path.join(BASE_DATA_PATH, "raw")
CLEAN_DATA_PATH = os.path.join(BASE_DATA_PATH, "clean")

### 01 Data Download
In this section we 
* load raw data into a dataframe
* peek into the first 5 rows

In [3]:
if not os.path.exists(RAW_DATA_PATH):
    os.makedirs(RAW_DATA_PATH)

In [4]:
df = stsbenchmark.load_pandas_df(RAW_DATA_PATH, file_split="train")
df.head()

100%|██████████| 401/401 [00:01<00:00, 310KB/s]  


Data downloaded to ../../data/raw/raw/stsbenchmark


,column_0,column_1,column_2,column_3,column_4,column_5,column_6
0,main-captions,MSRvid,2012test,0001,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,0004,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,0005,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,0006,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,0009,4.25,A man is playing the cello.,A man seated is playing the cello.


### 02 Data Cleaning
Now that we know about the general shape of the data, we can clean it so that it is ready for further preprocessing. The main operation we need for the STS Benchmark data is to drop all of columns except for the sentence pairs and scores.

In [5]:
sts_train = stsbenchmark.clean_sts(df)
sts_train.head()

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


### 03 Make Lowercase
We do simple standardization of the text by making all text lowercase.

In [6]:
sts_train_low = to_lowercase(sts_train)
sts_train_low.head()

,score,sentence1,sentence2
0,5.00,a plane is taking off.,an air plane is taking off.
1,3.80,a man is playing a large flute.,a man is playing a flute.
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...
3,2.60,three men are playing chess.,two men are playing chess.
4,4.25,a man is playing the cello.,a man seated is playing the cello.


### 04 Tokenize
We tokenize the text using spaCy's non-destructive tokenizer.

In [7]:
sts_train_tok = to_spacy_tokens(sts_train_low.head())
sts_train_tok.head()

,score,sentence1,sentence2,sentence1_tokens,sentence2_tokens
0,5.00,a plane is taking off.,an air plane is taking off.,"[a, plane, is, taking, off, .]","[an, air, plane, is, taking, off, .]"
1,3.80,a man is playing a large flute.,a man is playing a flute.,"[a, man, is, playing, a, large, flute, .]","[a, man, is, playing, a, flute, .]"
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...,"[a, man, is, spreading, shreded, cheese, on, a...","[a, man, is, spreading, shredded, cheese, on, ..."
3,2.60,three men are playing chess.,two men are playing chess.,"[three, men, are, playing, chess, .]","[two, men, are, playing, chess, .]"
4,4.25,a man is playing the cello.,a man seated is playing the cello.,"[a, man, is, playing, the, cello, .]","[a, man, seated, is, playing, the, cello, .]"


### 05 Optional: Remove Stop Words
Removing stop words is another common preprocessing step for NLP tasks. We use the `rm_spacy_stopwords` utility function to do this on the dataframe. This function makes use of the spaCy language model's default set of stop words. If we need to add our own set of stop words (for example, if we are doing an NLP task for a very specific domain of content), we can do this in-line by simply providing the list as the `custom_stopwords` parameter of `rm_spacy_stopwords`.

In [8]:
rm_spacy_stopwords(sts_train_tok).head()

,score,sentence1,sentence2,sentence1_tokens,sentence2_tokens,sentence1_tokens_rm_stopwords,sentence2_tokens_rm_stopwords
0,5.00,a plane is taking off.,an air plane is taking off.,"[a, plane, is, taking, off, .]","[an, air, plane, is, taking, off, .]","[plane, taking, .]","[air, plane, taking, .]"
1,3.80,a man is playing a large flute.,a man is playing a flute.,"[a, man, is, playing, a, large, flute, .]","[a, man, is, playing, a, flute, .]","[man, playing, large, flute, .]","[man, playing, flute, .]"
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...,"[a, man, is, spreading, shreded, cheese, on, a...","[a, man, is, spreading, shredded, cheese, on, ...","[man, spreading, shreded, cheese, pizza, .]","[man, spreading, shredded, cheese, uncooked, p..."
3,2.60,three men are playing chess.,two men are playing chess.,"[three, men, are, playing, chess, .]","[two, men, are, playing, chess, .]","[men, playing, chess, .]","[men, playing, chess, .]"
4,4.25,a man is playing the cello.,a man seated is playing the cello.,"[a, man, is, playing, the, cello, .]","[a, man, seated, is, playing, the, cello, .]","[man, playing, cello, .]","[man, seated, playing, cello, .]"
